# Drainage stability

When simulating, the head may cross the elevation of the tile drains or ground surface. When this happens de drainage resistance changes from `infinity` to `c_drain` when the head rises or from `c_drain` to `infinity` when the head is descending.
To deal with this in a managed fashion, when this happens during a time step, the moment at which this occurs is computed and the head is simulated up to the point at which it touches the drain elevation. At that point, the resistance is changed and the simulation is resumed until the end of the timestep is reached. The idea is that this yields a stable simulation, without sawtoothing.

The implementation, however, stills shows sawtoothing after the head hits the drainage level. This workbook tries to find out why this unexpected behavior occurs and seeks a way to prevent it.

@TO 2020-09-16

The governing partial differential equation

$$kD\frac{d^{2}h}{dx^{2}}	=-N+\frac{h-\phi}{c}+\frac{h-z_{dr}}{c_{dr}}$$

is equivalent to 

$$kD\frac{d^{2}h}{dx^{2}}=-N+\frac{h-\hat{\phi}}{\hat{c}}$$

with

$$\hat{c}=\frac{c_{dr}c}{c_{dr}+c}$$

and

$$\hat{\phi}=\frac{c_{dr}\phi+cz_{dr}}{c_{dr}+c}$$

Therefore, the solution for the transient approach followd in GGOR is

$$\overline{h}-\widehat{\phi}=\left(\overline{h}_{0}-\widehat{\phi}\right)e^{-\frac{t-t_{0}}{\widehat{T}}}+\left\{ N\widehat{c}-\widehat{\Lambda}\left[N\widehat{c}-\left(h_{LR}-\widehat{\phi}\right)\right]\right\} \left(1-e^{-\frac{t-t_{0}}{\widehat{T}}}\right)$$

with

$$\widehat{T}=\mu\widehat{c}$$

and

$$\widehat{\Lambda}=\frac{1}{\frac{b}{\widehat{\lambda}}\mbox{ctanh}\frac{b}{\widehat{\lambda}}+\frac{b/\widehat{c}}{D/w}}$$

In [10]:
import matplotlib.pyplot as plt
import numpy as np

In [20]:
# parameters
N, E, q = 0.002, 0., 0.001
h0, phi, hLR = 0., 0.5, 0.5
b, mu, k, D, c, cdr, zdr, w = 75., 0.15, 10., 10., 150, 5, 1.0, 1.0

h = h0

ch = cdr * c / (cdr + c)
phih = (cdr * phi + c * zdr) / (cdr + c)
Th = mu * ch
lamh = np.sqrt(k * D * ch)
Lamh = 1 / (b / lamh / np.tanh(b / lamh)  + b / ch / (D / w))
B = N * ch - (N * ch - (hLR - phih)) * Lamh

In [21]:
tau = Th * np.log((h0 - phih - B) / (zdr - phih - B))
print('tau = {}'.format(tau))

rising = ((phih - h) + (N * ch - (N * ch - (hLR - phih)) * Lamh)) > 0
print('rising = {}'.format(rising))


tau = nan
rising = True


/Users/Theo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


In [22]:
np.log((h0 - phih - B) / (zdr - phih - B))

/Users/Theo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


nan

In [30]:
(h0 - phih - B) / (zdr - phih - B)

-8.450276944629513

In [36]:
cdr = 0.0000000000000001
kD = 100
ch = c * cdr / (c + cdr)
lamh = np.sqrt(kD * ch)
b / lamh / np.tanh(b / lamh)

750000000.0